In [4]:
import pandas as pd 
import numpy as np
import os
import datetime
import re
import subprocess

# FIRST EXPERIMENTS: diff nodes and bandwidth

The experiments with 4-8-16 nodes, for No bandwidth restriction and 1Gbs restriction.
The threads are wrong there (16 instead of 64). So the results might be different compared to the other experiments.

The directory structure bellow.
We ignore the folders with: *to_sort, to_check, diff_splits*
```
.
|-- 1Gbs
|   |-- 16
|   |   |-- baseline
|   |   |   `-- 1
|   |   |       `-- prometheus
|   |   `-- rewritten
|   |       `-- 1
|   |           |-- prometheus_to_sort
|   |           `-- split_8
|   |-- 4
|   |   |-- baseline
|   |   |   `-- 1
|   |   |       `-- prometheus
|   |   `-- rewritten
|   |       `-- 1
|   |           `-- prometheus
|   `-- 8
|       |-- baseline
|       |   `-- 1
|       |       `-- prometheus
|       `-- rewritten
|           `-- 1
|               `-- prometheus
|-- No
|   |-- 16
|   |   |-- baseline
|   |   |   `-- 1
|   |   |       `-- prometheus
|   |   `-- rewritten
|   |       |-- 1
|   |       |   |-- diff_splits
|   |       |   `-- prometheus_so_sort
|   |       `-- 2
|   |           `-- prometheus
|   |-- 4
|   |   |-- baseline
|   |   |   `-- 1
|   |   |       `-- prometheus
|   |   `-- rewritten
|   |       |-- 1
|   |       |   `-- prometheus
|   |       `-- 2
|   |           `-- prometheus
|   `-- 8
|       |-- baseline
|       |   `-- 1
|       |       `-- prometheus
|       `-- rewritten
|           |-- 1
|           |   `-- prometheus
|           `-- 2
|               |-- prometheus
|               `-- prometheus_to_check
```

In [26]:
# get all data in one dataframe

bandwidths = ['No', '1Gbs'] 
nodes = [4, 8, 16] 
replicates_number = 2

df_results = pd.DataFrame()
prefix_path = "/home/ninon/description_prototype_stuff/merging_results/"
reports_files = []

for i in range(len(bandwidths)):
    for j in range(len(nodes)):
        for k in range(replicates_number):
            mode = "baseline"
            path = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(k+1)
            prefix = "_trace"
            #print(path)
            if os.path.exists(path):
                csv_files = [file for file in os.listdir(path) if file.startswith(prefix)]
                for file in csv_files:
                    file_path = os.path.join(path, file) 
                    df = pd.read_csv(file_path, delimiter="\t")
                    df['bandwidth'] = str(bandwidths[i])
                    df["node"] = str(nodes[j]) 
                    df["mode"] = mode 
                    df["replicate"] =  str(k+1)
                    df_results = df_results.append(df, ignore_index=True)
            else:
                mode = "rewritten"
                path = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(k+1)
                prefix = "_trace"
                #print(path)
                if os.path.exists(path):
                    csv_files = [file for file in os.listdir(path) if file.startswith(prefix)]
                    for file in csv_files:
                        file_path = os.path.join(path, file) 
                        df = pd.read_csv(file_path, delimiter="\t")
                        df['bandwidth'] = str(bandwidths[i])
                        df["node"] = str(nodes[j]) 
                        df["mode"] = mode 
                        df["replicate"] =  str(k+1)
                        df_results = df_results.append(df, ignore_index=True)
                else:
                    #print("no")
                    continue


columns_to_remove = ['native_id', 'status', 'exit', 'tag', 'attempt', 'queue', 'scratch', 'error_action', 'task_id', 'container', 'workdir']
df_results = df_results.drop(columns=columns_to_remove)

#nf resources report
columns_to_remove = ['disk', 'memory', 'rss', 'vmem','%cpu', '%mem', 'peak_rss', 'peak_vmem', 'rchar', 'wchar', 'syscr', 'syscw', 'read_bytes', 'write_bytes', 'vol_ctxt', 'inv_ctxt']
df_results = df_results.drop(columns=columns_to_remove)

print(df_results.head())



        hostname       hash                process  \
0  hu-worker-c27  90/a3cc1f        GFFREAD_TX2GENE   
1  hu-worker-c24  f2/938a2e   CUSTOM_GETCHROMSIZES   
2  hu-worker-c25  44/547e6d      DEXSEQ_ANNOTATION   
3  hu-worker-c24  8c/886f80  SALMON_GENOMEGENERATE   
4  hu-worker-c26  c3/1920aa    STAR_GENOMEGENERATE   

                                                name module  cpus time  \
0  GFFREAD_TX2GENE (Drosophila_melanogaster.BDGP6...      -     1    -   
1                               CUSTOM_GETCHROMSIZES      -     1    -   
2  DEXSEQ_ANNOTATION (Drosophila_melanogaster.BDG...      -     1    -   
3                              SALMON_GENOMEGENERATE      -     1    -   
4                                STAR_GENOMEGENERATE      -     1    -   

                    submit                    start                 complete  \
0  2024-04-25 13:56:07.089  2024-04-25 13:56:10.000  2024-04-25 13:56:14.000   
1  2024-04-25 13:56:06.653  2024-04-25 13:56:17.000  2024-04-25 13:56:

In [27]:
# Find NaN values

nan_rows = df[df.isna().any(axis=1)]
nan_counts_per_row = nan_rows.isna().sum(axis=1)

print("Rows with NaN values:")
print(nan_rows)
print("\nNumber of NaN values in each row:")
print(nan_counts_per_row)

Rows with NaN values:
Empty DataFrame
Columns: [hostname, task_id, hash, native_id, process, tag, name, status, exit, module, container, cpus, time, disk, memory, attempt, submit, start, complete, duration, realtime, queue, %cpu, %mem, rss, vmem, peak_rss, peak_vmem, rchar, wchar, syscr, syscw, read_bytes, write_bytes, vol_ctxt, inv_ctxt, env, workdir, scratch, error_action, bandwidth, node, mode, replicate]
Index: []

[0 rows x 44 columns]

Number of NaN values in each row:
Series([], dtype: int64)


now we isolate the tasks that were replaced or split:
- STAR_GENOMEGENERATE by HISAT2_INDEX
- STAR_ALIGN replaced by HISAT2_ALIGN
- SAMTOOLS (split)
- FASTQSPLIT_HISAT2:SPLIT_READS (added befiore splitted tasks)
- SAMTOOLS_MERGE_HISAT2 (added befiore samtools tasks)

In [23]:
list_of_tasks_that_were_modified = ["STAR_GENOMEGENERATE", "HISAT2_INDEX", "STAR_ALIGN", "HISAT2_ALIGN", "SAMTOOLS", "FASTQSPLIT_HISAT2", "SAMTOOLS_MERGE_HISAT2"]
filtered_df = df_results[df_results['name'].isin(list_of_tasks_that_were_modified)]

print(filtered_df)

          hostname       hash              process                 name  \
4    hu-worker-c26  c3/1920aa  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
59   hu-worker-c24  fc/27be8c         HISAT2_INDEX         HISAT2_INDEX   
116  hu-worker-c26  7c/95122c  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
171  hu-worker-c43  e0/fb5948         HISAT2_INDEX         HISAT2_INDEX   
292  hu-worker-c39  0a/356982  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
347  hu-worker-c27  27/40a28c         HISAT2_INDEX         HISAT2_INDEX   
532  hu-worker-c25  1e/bce78f  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
588  hu-worker-c25  2c/d48e1f  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
644  hu-worker-c38  5a/133a2c  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   

    module  cpus time                   submit                    start  \
4        -     1    -  2024-04-25 13:56:06.742  2024-04-25 13:56:16.000   
59       -     1    -  2024-04-28 15:19:20.251  2024-04-28 15:19:22.000   
116      -     1    -  2

In [25]:

# Group the DataFrame by 'nodes' and 'bandwidth'
grouped_df = filtered_df.groupby(['node', 'bandwidth'])

# Iterate over the groups
for group, group_df in grouped_df:
    print("Group:", group)
    print(group_df)
    print()

Group: ('16', '1Gbs')
          hostname       hash              process                 name  \
644  hu-worker-c38  5a/133a2c  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   

    module  cpus time                   submit                    start  \
644      -     1    -  2024-04-27 16:15:21.048  2024-04-27 16:15:42.000   

                    complete duration realtime env bandwidth node      mode  \
644  2024-04-27 16:19:52.000   4m 31s    4m 7s   -      1Gbs   16  baseline   

    replicate  
644         1  

Group: ('16', 'No')
          hostname       hash              process                 name  \
292  hu-worker-c39  0a/356982  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
347  hu-worker-c27  27/40a28c         HISAT2_INDEX         HISAT2_INDEX   

    module  cpus time                   submit                    start  \
292      -     1    -  2024-04-26 10:06:55.254  2024-04-26 10:06:58.000   
347      -     1    -  2024-04-28 20:51:20.408  2024-04-28 20:51:22.000   

            

From there, you can query the traces from prometheus. 
- The column "hostname" is giving you the node a task was executed on
- the column "start" gives you the start time
- the column "complete" gives you the endtime of a task

TODO VASILIS

In [30]:
# TODO: 
# - for each condition, compare the baseline to the rewritten (using the start/end time and hostname)
# - see what metrics are very influenced by the rewritting (get percentage increase/reduction compared to baseline?)
# - see how the number of nodes and bandwidth restriction influenced the results (if it is the case)
# - generate a few fancy graphs that answer / illustrate well the questions above

# Experiment 2: individual effects of different adaptations

there we study:
- only replace,
- only split
- only threads
- baseline
- rewritten

```
rnasplice_exp_traces_adapt
|-- command_baseline
|   |-- 0
|   `-- 1
|-- command_replace
|   |-- 0
|   `-- 1
|-- command_rewritting_full
|   |-- 0
|   `-- 1
|-- command_split
|   |-- 0
|   `-- 1
`-- command_threads
    |-- 0
    `-- 1

```

In [43]:

replicates_number = 2

experiments = ["command_baseline", "command_replace", "command_threads", "command_split", "command_rewritting_full"]

df_adapt = pd.DataFrame()
prefix_path = "/home/ninon/description_prototype_stuff/merging_results/rnasplice_exp_traces_adapt"
reports_files = []

for i in range(len(experiments)):
    for k in range(replicates_number):
        path = prefix_path + "/"+ str(experiments[i]) + "/"+ str(k+1)
        prefix = "_trace"
        #print(path)
        if os.path.exists(path):
            csv_files = [file for file in os.listdir(path) if file.startswith(prefix)]
            for file in csv_files:
                file_path = os.path.join(path, file) 
                df = pd.read_csv(file_path, delimiter="\t")
                df['experiment'] = str(experiments[i])
                df["replicate"] =  str(k+1)
                df_adapt = df_adapt.append(df, ignore_index=True)
            else:
                #print("no")
                continue

print(df_adapt.head())

columns_to_remove = ['native_id', 'status', 'exit', 'tag', 'attempt', 'queue', 'scratch', 'error_action', 'task_id', 'container', 'workdir']
df_adapt = df_adapt.drop(columns=columns_to_remove)

#nf resources report
columns_to_remove = ['disk', 'memory', 'rss', 'vmem','%cpu', '%mem', 'peak_rss', 'peak_vmem', 'rchar', 'wchar', 'syscr', 'syscw', 'read_bytes', 'write_bytes', 'vol_ctxt', 'inv_ctxt']
df_adapt = df_adapt.drop(columns=columns_to_remove)

print(df_adapt.head())



        hostname  task_id       hash                            native_id  \
0  hu-worker-c23        3  89/b6379c  nf-89b6379c0a8b9d3e50f759e6deb34eac   
1  hu-worker-c29        5  fb/8ac881  nf-fb8ac881e14b84e1176b47f6ad1c0667   
2  hu-worker-c30        2  17/dbcf73  nf-17dbcf73f32c776801487a0f9179c2f0   
3  hu-worker-c36        1  da/8a5d25  nf-da8a5d2563a12b7e3904c3c3ffef76b8   
4  hu-worker-c29        4  ab/ac93b4  nf-abac93b4822e7bbbc17273da2bfddb95   

                 process                                       tag  \
0   CUSTOM_GETCHROMSIZES                                         -   
1        GFFREAD_TX2GENE  Drosophila_melanogaster.BDGP6.32.106.gtf   
2      DEXSEQ_ANNOTATION  Drosophila_melanogaster.BDGP6.32.106.gtf   
3  SALMON_GENOMEGENERATE                                         -   
4    STAR_GENOMEGENERATE                                         -   

                                                name     status  exit module  \
0                               CUST

In [41]:
# Find NaN values

nan_rows = df_adapt[df_adapt.isna().any(axis=1)]
nan_counts_per_row = nan_rows.isna().sum(axis=1)

print("Rows with NaN values:")
print(nan_rows)
print("\nNumber of NaN values in each row:")
print(nan_counts_per_row)

Rows with NaN values:
Empty DataFrame
Columns: [hostname, hash, process, name, module, cpus, time, submit, start, complete, duration, realtime, env, experiment, replicate]
Index: []

Number of NaN values in each row:
Series([], dtype: int64)


now we isolate the tasks that were replaced or split:
- STAR_GENOMEGENERATE by HISAT2_INDEX
- STAR_ALIGN replaced by HISAT2_ALIGN
- SAMTOOLS (split)
- FASTQSPLIT_HISAT2:SPLIT_READS (added befiore splitted tasks)
- SAMTOOLS_MERGE_HISAT2 (added befiore samtools tasks)

In [42]:
list_of_tasks_that_were_modified = ["STAR_GENOMEGENERATE", "HISAT2_INDEX", "STAR_ALIGN", "HISAT2_ALIGN", "SAMTOOLS", "FASTQSPLIT_HISAT2", "SAMTOOLS_MERGE_HISAT2"]
filtered_df = df_adapt[df_adapt['name'].isin(list_of_tasks_that_were_modified)]

print(filtered_df)

          hostname       hash              process                 name  \
4    hu-worker-c29  ab/ac93b4  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
59   hu-worker-c30  3c/237396         HISAT2_INDEX         HISAT2_INDEX   
115  hu-worker-c36  e3/e5c8a8         HISAT2_INDEX         HISAT2_INDEX   
236  hu-worker-c29  ae/a172e4  STAR_GENOMEGENERATE  STAR_GENOMEGENERATE   
299  hu-worker-c29  61/fce0bb         HISAT2_INDEX         HISAT2_INDEX   

    module  cpus time                   submit                    start  \
4        -     1    -  2024-05-08 09:29:35.429  2024-05-08 09:29:44.000   
59       -     1    -  2024-05-08 19:23:05.995  2024-05-08 19:23:10.000   
115      -     1    -  2024-05-08 23:24:22.483  2024-05-08 23:24:16.000   
236      -     1    -  2024-05-09 02:09:25.858  2024-05-09 02:09:15.000   
299      -     1    -  2024-05-09 12:58:49.895  2024-05-09 12:58:57.000   

                    complete duration realtime env               experiment  \
4    2024-05-08 09:

TODO VASILIS

In [34]:
# TODO: 
# - compare the different experiments (using the start/end time and hostname)
# - see what metrics are very influenced by the different rewritting (get percentage increase/reduction compared to baseline?): example replace saves ram by 8% but increases CPU usage by 12%
# - generate a few fancy graphs